In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

import tensorflow as tf


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop



sns.set_theme(style='white', context='notebook', palette='deep')

In [3]:
train_X = pd.read_csv("kaggle_datasets/train.csv")
test_data = pd.read_csv("kaggle_datasets/test.csv")
train_Y = train_X.pop('label')

In [4]:
from keras.utils import to_categorical
#train_X.describe()
train_X = train_X / 255.0
test_data = test_data / 255.0
train_X = train_X.values.reshape(-1,28,28,1)
test_data = test_data.values.reshape(-1,28,28,1)
train_Y = to_categorical(train_Y, num_classes = 10)

In [5]:
from sklearn.model_selection import train_test_split
#display(train_X.shape, test_data.shape)
train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size = 0.1, random_state=2)

In [33]:
# g = plt.imshow(train_X[0][:,:,0])
# display(train_X[0][:,:,0])

In [6]:
model = Sequential([
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)),
    Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation = "relu"),
    Dropout(0.5),
    Dense(10, activation = "softmax")
])
# model = Sequential([
#     Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
#     Conv2D(64, kernel_size=(3, 3), activation='relu'),
#     MaxPool2D(pool_size=(2, 2)),
#     Dropout(0.25),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(10, activation='softmax')
# ])

In [35]:
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

In [36]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(train_X)


In [37]:
history = model.fit(datagen.flow(train_X,train_Y, batch_size=86),
                              epochs = 40, validation_data = (val_X,val_Y),
                              verbose = 2, steps_per_epoch=train_X.shape[0] // 86
                              , callbacks=[learning_rate_reduction])

Epoch 1/40


c:\Users\ianmm\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


439/439 - 18s - 41ms/step - accuracy: 0.8720 - loss: 0.4054 - val_accuracy: 0.9769 - val_loss: 0.0736 - learning_rate: 0.0010
Epoch 2/40


c:\Users\ianmm\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\callbacks\callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_acc` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
c:\Users\ianmm\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


439/439 - 1s - 2ms/step - accuracy: 0.9884 - loss: 0.0944 - val_accuracy: 0.9802 - val_loss: 0.0697 - learning_rate: 0.0010
Epoch 3/40
439/439 - 16s - 37ms/step - accuracy: 0.9626 - loss: 0.1217 - val_accuracy: 0.9879 - val_loss: 0.0420 - learning_rate: 0.0010
Epoch 4/40
439/439 - 1s - 2ms/step - accuracy: 0.9651 - loss: 0.1588 - val_accuracy: 0.9838 - val_loss: 0.0518 - learning_rate: 0.0010
Epoch 5/40
439/439 - 16s - 37ms/step - accuracy: 0.9734 - loss: 0.0916 - val_accuracy: 0.9907 - val_loss: 0.0297 - learning_rate: 0.0010
Epoch 6/40
439/439 - 1s - 2ms/step - accuracy: 0.9884 - loss: 0.0340 - val_accuracy: 0.9893 - val_loss: 0.0327 - learning_rate: 0.0010
Epoch 7/40
439/439 - 16s - 37ms/step - accuracy: 0.9770 - loss: 0.0776 - val_accuracy: 0.9912 - val_loss: 0.0292 - learning_rate: 0.0010
Epoch 8/40
439/439 - 1s - 2ms/step - accuracy: 0.9767 - loss: 0.1189 - val_accuracy: 0.9898 - val_loss: 0.0303 - learning_rate: 0.0010
Epoch 9/40
439/439 - 16s - 37ms/step - accuracy: 0.9800 - lo

In [7]:
# import tensorflowjs as tfjs
# tfjs.converters.save_keras_model(model, 'model.json')

# model.save('model.keras')

# # Convert the Keras model to TensorFlow.js format
!tensorflowjs_converter --input_format keras model.keras model.json


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\ianmm\AppData\Local\Programs\Python\Python311\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "C:\Users\ianmm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "C:\Users\ianmm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflowjs\converters\__init__.py", line 21, in <module>
    from tensorflowjs.converters.converter import convert
  File "C:\Users\ianmm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflowjs\converters\converter.py", line 38, in <module>
    from tensorflowjs.converters import tf_saved_model_conversion_v2
  File "C:\Users\ianmm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflowjs\converters\tf_saved_model_conversion_v2.py", line 28, in

In [39]:
# results = model.predict(test_data)

# # select the indix with the maximum probability
# results = np.argmax(results,axis = 1)

# results = pd.Series(results,name="Label")

In [40]:
# submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

# submission.to_csv("submission.csv",index=False)

In [41]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "digit_classification.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)